**Read
- https://www.jittagornp.me/blog/what-is-firewall/

Columns Description
- Source Port, Client Source Port 
- Destination Port, Client Destination Port 
- NAT Source Port, Network Address Translation Source Port 
- NAT Destination Port, Network Address Translation Destination Port 
- Elapsed Time (sec), Elapsed Time for flow 
- Bytes, Total Bytes 
- Bytes Sent, Bytes Sent 
- Bytes Received, Bytes Received 
- Packets, Total Packets 
- pkts_sent, Packets Sent 
- pkts_received, Packets Received
- Action, Class (allow, deny, drop, reset-both)

- Allow, Allows the internet traffic. 
- Deny, Blocks traffic and enforces the default Deny Action defined for the application that is being denied. 
- Drop, Silently drops the traffic; for an application, it overrides the default deny action. A TCP reset is not sent to the host/application. 
- Reset-Both, Sends a TCP reset to both the client-side and server-side devices

NAT stands for network address translation. It’s a way to map multiple private addresses inside a local network to a public IP address before transferring the information onto the internet. Organizations that want multiple devices to employ a single IP address use NAT, as do most home routers

Idea
- คิดว่า feature packet ทั้งหมด drop ทิ้งได้ เพราะ  corr กับ bytes
- ทำรูป flow pipeline 

To do
- encoding catagorical data
- dicision tree adjust branch
- random forest
- knn
- xgboost
- cart
- handle imbalance data
    - upsampling with smote make perf getting worse

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

In [ ]:
# Import data
df = pd.read_csv('Firewall_Log.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.describe()

In [ ]:
# count the occurrences of each unique value
action_counts = df['Action'].value_counts()

# plot the count of unique values
ax = action_counts.plot(kind='bar')
ax.set_title('Count of Action Values')

for i in ax.patches:
    ax.text(i.get_x()+0.1, i.get_height(), str(int(i.get_height())), fontsize=11, color='black')
plt.show()

# pie chart
action_counts.plot.pie(autopct='%.2f')
plt.show()

มี imbalance data ใน action == reset-both

In [ ]:
# from sklearn.utils import resample


# allow = df[df.Action=='allow']
# deny = df[df.Action=='deny']
# drop = df[df.Action=='drop']
# reset_both = df[df.Action=='reset-both']

# # oversample the minority classes
# deny_oversampled = resample(deny, replace=True, n_samples=len(allow), random_state=42)
# drop_oversampled = resample(drop, replace=True, n_samples=len(allow), random_state=42)
# reset_both_oversampled = resample(reset_both, replace=True, n_samples=len(allow), random_state=42)

# # combine the resampled classes with the majority class
# oversampled_df = pd.concat([allow, deny_oversampled, drop_oversampled, reset_both_oversampled])

# # check the class distribution
# oversampled_df['Action'].value_counts()

In [ ]:
# Convert port column to str type
col = ['Source Port', 'Destination Port', 'NAT Source Port', 'NAT Destination Port']
for i in col:
    df[i] = df[i].astype(str)

In [ ]:
fig, axs = plt.subplots(ncols=7, figsize=(15, 3))

for i, col in enumerate(['Bytes', 'Bytes Sent', 'Bytes Received', 'Packets', 'pkts_sent', 'pkts_received', 'Elapsed Time (sec)']):
    action_median = df.groupby(['Action'])[col].median()
    action_median.plot(kind='bar', ax=axs[i])
    axs[i].set_title(col)

plt.tight_layout()
plt.show()

In [ ]:
# create histogram
columns = ['Bytes', 'Bytes Sent', 'Bytes Received', 'Packets', 'Elapsed Time (sec)', 'pkts_sent', 'pkts_received']

fig, axs = plt.subplots(nrows=3, ncols=3, figsize=(8, 6))
axs = axs.flatten()

# Create a histogram for each selected column on a separate subplot
for i, col in enumerate(columns):
    axs[i].hist(df[col], bins=20)
    axs[i].set_title(col)

plt.tight_layout()
plt.show()

In [ ]:
# Since the data in columns is not form in normal distribution so we will apply Standardization
scaler = StandardScaler()
df[columns] = scaler.fit_transform(df[columns])

df.head()

In [ ]:
df.loc[df['Action']=='reset-both'].head()

In [ ]:
# df['S-D_Port'] = df['Source Port'] + '_' + df['Destination Port']
# df.head()

In [ ]:
# Create a heatmap of the correlation coefficients
corr_matrix = df[['Bytes', 'Bytes Sent', 'Bytes Received', 'Packets', 'Elapsed Time (sec)', 'pkts_sent', 'pkts_received']].corr()

sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.show()

# packets และ bytes corr=~1

In [ ]:
df = pd.get_dummies(df, columns=['Source Port', 'Destination Port', 'NAT Source Port', 'NAT Destination Port'])
df

In [ ]:
X = df.drop(['Action', 'Packets', 'pkts_sent', 'pkts_received'], axis=1)
y = df['Action']

In [ ]:
# from sklearn import preprocessing
# le = preprocessing.LabelEncoder()
# label_encoded_df = X.copy()
# for col in label_encoded_df.select_dtypes(include='O').columns:
#     label_encoded_df[col]=le.fit_transform(label_encoded_df[col])

In [ ]:
# from sklearn import preprocessing
# le = preprocessing.LabelEncoder()
# for col in X.select_dtypes(include='O').columns:
#     X[col]=le.fit_transform(df[col])

In [ ]:
# X = df.drop(['Action', 'Bytes Sent', 'Bytes Received', 'pkts_sent', 'pkts_received'], axis=1)
# y = df['Action']

In [ ]:
# X = df.drop(['Action', 'Elapsed Time (sec)'], axis=1)
# y = df['Action']

In [ ]:
# from sklearn.preprocessing import OneHotEncoder

# # create a list of column names to encode
# cols_to_encode = ['Source Port', 'Destination Port', 'NAT Source Port', 'NAT Destination Port']

# # create a OneHotEncoder object
# ohe = OneHotEncoder(sparse_output=False)
# ohe.fit_transform(df[['Source Port']])

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(X,y,train_size=0.8, test_size=0.2, random_state=50)

In [ ]:
treeclassifier = DecisionTreeClassifier()
treeclassifier.fit(X_train, y_train)
pred = treeclassifier.predict(X_test)

In [ ]:
# handle imbalance data (reset-both) with SMOTE
# ลองแล้วใน disition tree perf แย่กว่าเดิม
# แย่น่าจะเพราะ smote ใช้ knn upsampling และเลขค่า port ที่ generate จะเปลี่ยนจากเดิมทำให้ perf แย่ลง เพราะ ค่า port สำคัญในการหา y

# print('Original dataset shape %s' % Counter(y_train))

sm = SMOTE(random_state=1)
X_sm, y_sm = sm.fit_resample(X_train, y_train)
# print('Resampled dataset shape %s' % Counter(y_sm))

y_sm.value_counts()

In [ ]:
# # ลอง round ค่า port จาก smote ดูแย่เหมือนเดิม
# cols = ['Source Port', 'Destination Port', 'NAT Source Port', 'NAT Destination Port']
# for col in cols:
#     X_sm[col] = X_sm[col].apply(round)
# X_sm.head()

In [ ]:
# treeclassifier.fit(X_sm, y_sm)
# pred = treeclassifier.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))

# KNN

In [ ]:
import math
max_k = round(math.sqrt(len(X_train)))
print(f"maximun k = {max_k}")

In [ ]:
# from sklearn.neighbors import KNeighborsClassifier


# test_scores = []
# train_scores = []

# for i in range(1, max_k+1):

#     knn = KNeighborsClassifier(i)
#     knn.fit(X_train,y_train)
    
#     train_scores.append(knn.score(X_train,y_train))
#     test_scores.append(knn.score(X_test,y_test))

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(["paris", "paris", "tokyo", "amsterdam"])